In [1]:
import os
import requests
import zipfile
import pathlib

In [2]:
import torch
import torch.nn as nn
import torchvision

In [3]:
data_path = pathlib.Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
  print(f"{image_path} directory exist.")
else:
  image_path.mkdir(parents=True, exist_ok=True)
  print(f"{image_path} directory created.")

with open(data_path/"pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza, steak, sushi data...")
  f.write(request.content)

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping pizza, steak, sushi data...")
  zip_ref.extractall(image_path)

data/pizza_steak_sushi directory created.
Unzipping pizza, steak, sushi data...


In [5]:
%%writefile going_modular/data_setup.py
import os
import torch
import torchvision

NUM_WORKERS = os.cpu_count()
def create_dataLoader(
    train_dir,
    test_dir,
    transform,
    batch_size,
    num_workers : int = NUM_WORKERS
):
  """
  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.
  """
  train_set = torchvision.datasets.ImageFolder(train_dir, transform)
  test_set = torchvision.datasets.ImageFolder(test_dir, transform)

  train_dataLoader = torch.utils.data.DataLoader(dataset = train_set, batch_size = batch_size, num_workers = num_workers, pin_memory = True, shuffle = True)
  test_dataLoader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, num_workers = num_workers, pin_memory = True, shuffle = False)

  classes = train_set.classes

  return train_dataLoader, test_dataLoader, classes

Writing going_modular/data_setup.py


In [6]:
%%writefile going_modular/model_build.py
import torch
from torch import nn

class TinyVGG(nn.Module):
  def __init__(self, input_shape, hidden_units, output_shape):
    super().__init__()
    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size = 3, stride = 1, padding = 0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size = 3, stride = 1, padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.conv_block2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size = 3, stride = 1, padding = 0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size = 3, stride = 1, padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*13*13, out_features = output_shape)
    )

  def forward(self,x):
    return self.classifier(self.conv_block2(self.conv_block1(x)))


Writing going_modular/model_build.py


In [7]:
%%writefile going_modular/train_test.py
import torch

def train_step(model, trainDataLoader, optimizer, loss_fn):
  model.train()
  total_train_loss = 0
  total_train_acc = 0
  for train_images, train_labels in trainDataLoader:
    optimizer.zero_grad()
    y_train_logits = model(train_images)
    train_loss = loss_fn(y_train_logits, train_labels)
    total_train_loss += train_loss.item()
    train_loss.backward()
    optimizer.step()
    y_train_pred = torch.argmax(y_train_logits, dim=1)
    total_train_acc += ((y_train_pred == train_labels).sum().item() / len(train_labels))
  total_train_loss = total_train_loss / len(trainDataLoader)
  total_train_acc = total_train_acc / len(trainDataLoader)

  return total_train_loss, total_train_acc

def test_step(model, testDataLoader, loss_fn):
  model.eval()
  total_test_loss = 0
  total_test_acc = 0
  with torch.inference_mode():
    for test_images, test_labels in testDataLoader:
      y_test_logits = model(test_images)
      test_loss = loss_fn(y_test_logits, test_labels)
      total_test_loss += test_loss.item()
      y_test_pred = torch.argmax(y_test_logits, dim = 1)
      total_test_acc += ((y_test_pred == test_labels).sum().item() / len(test_labels))
    total_test_loss = total_test_loss / len(testDataLoader)
    total_test_acc = total_test_acc / len(testDataLoader)

  return total_test_loss, total_test_acc

def train_model(model, trainDataLoader, testDataLoder, optimizer, loss_fn, epochs):
  results = {
      "train_loss" : [],
      "train_acc" : [],
      "test_loss" : [],
      "test_acc" : []
  }

  for epoch in range(epochs):
    train_loss, train_acc = train_step(model, trainDataLoader, optimizer, loss_fn)
    test_loss, test_acc = test_step(model, testDataLoder, loss_fn)
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results

Writing going_modular/train_test.py


In [8]:
%%writefile going_modular/utils.py
import torch
from pathlib import Path

def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
  """
      Args:
            model: A target PyTorch model to save.
            target_dir: A directory for saving the model to.
            model_name: A filename for the saved model. Should include either ".pth" or ".pt" as the file extension.
  """
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents = True, exist_ok = True)

  assert model_name.endswith(".pth") or model_name.endswith(".pt")
  model_save_path = target_dir_path / model_name
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj = model.state_dict(), f = model_save_path)

Writing going_modular/utils.py


In [9]:
%%writefile going_modular/train.py

import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from pathlib import Path
from timeit import default_timer as timer
import data_setup, model_build, train_test, utils

NUM_EPOCHS = 10
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

train_dir = Path("data/pizza_steak_sushi/train")
test_dir = Path("data/pizza_steak_sushi/test")

transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

train_dataLoader, test_DataLoader, classes = data_setup.create_dataLoader(train_dir,
                                                                          test_dir,
                                                                          transform,
                                                                          batch_size = BATCH_SIZE)

model_0 = model_build.TinyVGG(input_shape = 3, hidden_units = HIDDEN_UNITS, output_shape = len(classes))

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_0.parameters(), lr = LEARNING_RATE)

start_time = timer()
train_test.train_model(model_0, train_dataLoader, test_DataLoader, optimizer, loss_fn, NUM_EPOCHS)
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
utils.save_model(model_0, target_dir = "models", model_name = "05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train.py


In [10]:
!python going_modular/train.py

Epoch: 1, Train Loss: 1.0993, Train Acc: 0.3086 | Test Loss: 1.1117, Test Acc: 0.2917
Epoch: 2, Train Loss: 1.1031, Train Acc: 0.3047 | Test Loss: 1.1180, Test Acc: 0.1979
Epoch: 3, Train Loss: 1.0801, Train Acc: 0.5117 | Test Loss: 1.0781, Test Acc: 0.4129
Epoch: 4, Train Loss: 1.0378, Train Acc: 0.5820 | Test Loss: 1.0728, Test Acc: 0.4318
Epoch: 5, Train Loss: 1.0096, Train Acc: 0.5508 | Test Loss: 1.1111, Test Acc: 0.3125
Epoch: 6, Train Loss: 1.0658, Train Acc: 0.3906 | Test Loss: 1.0764, Test Acc: 0.2907
Epoch: 7, Train Loss: 0.9634, Train Acc: 0.6445 | Test Loss: 0.9994, Test Acc: 0.4631
Epoch: 8, Train Loss: 0.9057, Train Acc: 0.6250 | Test Loss: 0.9783, Test Acc: 0.4631
Epoch: 9, Train Loss: 1.0250, Train Acc: 0.5273 | Test Loss: 0.9746, Test Acc: 0.4328
Epoch: 10, Train Loss: 1.0015, Train Acc: 0.4766 | Test Loss: 0.9778, Test Acc: 0.4640
[INFO] Total training time: 17.666 seconds
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth
